# Depreciation to FERC 1 Connection

This stage in the connection process is very much under constuction

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import pudl
import pudl.constants as pc
import sqlalchemy as sa
import logging
import sys
import copy
import pathlib
import random
import warnings
from copy import deepcopy

import pudl_rmi.connect_deprish_to_eia
import pudl_rmi.make_plant_parts_eia
import pudl_rmi.connect_deprish_to_ferc1
import pudl_rmi.deprish
import pudl_rmi.coordinate

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

### Prepare the inputs

In [ ]:
file_path_mul = pathlib.Path().cwd().parent / 'outputs'/ 'master_unit_list.pkl.gz'
file_path_ferc1_eia = pathlib.Path().cwd().parent / 'outputs' /'ferc1_to_eia.pkl.gz'
file_path_deprish_eia = pathlib.Path().cwd().parent / 'outputs' /'deprish_to_eia.pkl.gz'
file_path_deprish = pathlib.Path().cwd().parent/'inputs'/'depreciation_rmi.xlsx'
file_path_deprish_ferc1 = pathlib.Path().cwd().parent / 'outputs' /'deprish_to_ferc1.pkl.gz'
file_path_training = pathlib.Path().cwd().parent /'inputs'/'train_ferc1_to_eia.csv'
# pudl output object
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])
pudl_out = pudl.output.pudltabl.PudlTabl(
    pudl_engine,freq='AS',
    fill_fuel_cost=True,
    roll_fuel_cost=True,
    fill_net_gen=False
)

In [ ]:
rmi_out = pudl_rmi.coordinate.Output(
    pudl_out,
    file_path_mul,
    file_path_deprish,
    file_path_deprish_eia,
    file_path_training,
    file_path_ferc1_eia,
    file_path_deprish_ferc1,
)

In [ ]:
clobber=False
plant_parts_eia = rmi_out.get_plant_part_list()
deprish_eia = rmi_out.get_deprish_to_eia()
ferc1_to_eia=rmi_out.get_ferc1_to_eia()

In [ ]:
inputs = InputsManager(
    plant_parts_eia=plant_parts_eia,
    deprish_eia=deprish_eia,
    ferc1_to_eia=ferc1_to_eia,
    clobber=clobber
)

scaler = Scaler(MatchMaker(inputs))
scaled_df = scaler.scale()

### Make Output in RMI Format

In [ ]:
file_path_deprish = pathlib.Path().cwd().parent/ 'inputs'/'depreciation_rmi.xlsx'
sheet_name_deprish='Depreciation Studies Raw'
transformer = pudl_rmi.deprish.Transformer(
    pudl_rmi.deprish.Extractor(
        file_path=file_path_deprish,
        sheet_name=sheet_name_deprish
    ).execute())
deprish_df = transformer.execute()
deprish_asset_df = pudl_rmi.deprish.agg_to_idx(
    deprish_df,
    idx_cols=[x for x in pudl_rmi.deprish.IDX_COLS_DEPRISH if x not in ['ferc_acct', 'note']])

In [ ]:
connected_deprish = pudl_rmi.connect_deprish_to_ferc1.rmi_output_ify(scaled_df, deprish_df)

### Export to output tables

In [ ]:
file_path_dc = pathlib.Path().cwd().parent /'outputs' /'deprish_w_common.csv.gz'
deprish_df.to_csv(file_path_dc)

In [ ]:
file_path_cd = pathlib.Path().cwd().parent /'outputs' /'deprish_to_ferc.csv.gz'
connected_deprish.to_csv(file_path_cd)

In [ ]:
file_path_da = pathlib.Path().cwd().parent /'outputs' /'deprish_w_common_by_asset.csv.gz'
deprish_asset_df.to_csv(file_path_da)

In [ ]:
file_path_scaled = pathlib.Path().cwd().parent /'outputs' /'ferc_to_deprish_scaled.csv.gz'
scaled_df.to_csv(file_path_scaled)